In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, BatchNormalization, Flatten, LSTM, Dense,Input
from sklearn.model_selection import train_test_split
import librosa
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def load_audio_files(data_folder):
    audio_signals = []
    labels = []
    class_folders = os.listdir(data_folder)
    for label, class_folder in enumerate(class_folders):
        class_path = os.path.join(data_folder, class_folder)
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            signal, sr = librosa.load(file_path, sr=None)

            # Normalize the audio signal
            min_val = np.min(signal)
            max_val = np.max(signal)
            signal = (signal - min_val) / (max_val - min_val)

            audio_signals.append(signal)
            labels.append(label)
    return audio_signals, labels

train_folder = '/content/drive/My Drive/Audios/Train'
validation_folder = '/content/drive/My Drive/Audios/Validation'
test_folder = '/content/drive/My Drive/Audios/Test'

train_signals, train_labels = load_audio_files(train_folder)
validation_signals, validation_labels = load_audio_files(validation_folder)
test_signals, test_labels = load_audio_files(test_folder)

# **CAE**

In [ ]:
classes = sorted(os.listdir(train_folder))

#in order the AE to process the signals easily, frame the signals into 25 ms
frame_length = 400 #25 window size
train_frames = [librosa.util.frame(signal, frame_length=frame_length, hop_length=frame_length).T for signal in train_signals]
val_frames = [librosa.util.frame(signal, frame_length=frame_length, hop_length=frame_length).T for signal in validation_signals]
test_frames = [librosa.util.frame(signal, frame_length=frame_length, hop_length=frame_length).T for signal in test_signals]

#Flatten frames for autoencoder
flat_X_train = np.concatenate(train_frames, axis=0)
flat_X_val = np.concatenate(val_frames, axis=0)
flat_X_test = np.concatenate(test_frames, axis=0)

y_train=train_labels
y_val=validation_labels
y_test=test_labels

In [ ]:
# Reshape for Conv1D input (add channel dimension)
from tensorflow.keras.layers import UpSampling1D
flat_X_train_reshaped = flat_X_train.reshape((flat_X_train.shape[0], flat_X_train.shape[1], 1))
flat_X_val_reshaped = flat_X_val.reshape((flat_X_val.shape[0], flat_X_val.shape[1], 1))
flat_X_test_reshaped = flat_X_test.reshape((flat_X_test.shape[0], flat_X_test.shape[1], 1))

# Define input shape based on reshaped data
input_shape = (flat_X_train_reshaped.shape[1], flat_X_train_reshaped.shape[2])

# Define Convolutional Autoencoder model
inputs = Input(shape=input_shape)
x = Conv1D(32, 3, activation='relu', padding='same')(inputs)
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(64, 3, activation='relu', padding='same')(x)
x = MaxPooling1D(2, padding='same')(x)

# Latent space
encoded = Conv1D(128, 3, activation='relu', padding='same')(x)

# Decoder
x = Conv1D(64, 3, activation='relu', padding='same')(encoded)
x = UpSampling1D(2)(x)
x = Conv1D(32, 3, activation='relu', padding='same')(x)
x = UpSampling1D(2)(x)
decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(x)

# Autoencoder model
autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

autoencoder.summary()

# Train the autoencoder
history = autoencoder.fit(flat_X_train_reshaped, flat_X_train_reshaped,
                          epochs=100, batch_size=32,
                          validation_data=(flat_X_val_reshaped, flat_X_val_reshaped),
                          verbose=1)

In [ ]:
encoder = Model(inputs, encoded)

In [ ]:
#save the model
autoencoder.save('/content/drive/My Drive/Results/Models/Cae.h5')
encoder.save('/content/drive/My Drive/Results/Models/CaeEncoder.h5')
encoded_X_train=encoder.predict(flat_X_train_reshaped)
encoded_X_val=encoder.predict(flat_X_val_reshaped)
encoded_X_test=encoder.predict(flat_X_test_reshaped)

In [ ]:
#save the encoded signals
import pickle

with open('/content/drive/My Drive/Results/Models/CaeTrainEncoded.pkl', 'wb') as f:
    pickle.dump((encoded_X_train, y_train), f)
with open('/content/drive/My Drive/Results/Models/CaeValEncoded.pkl', 'wb') as f:
    pickle.dump((encoded_X_val, y_val), f)
with open('/content/drive/My Drive/Results/Models/CaeTestEncoded.pkl', 'wb') as f:
    pickle.dump((encoded_X_test, y_test), f)

In [ ]:
new_train_shape=(int(encoded_X_train.shape[0]/20), 20, encoded_X_train.shape[1], encoded_X_train.shape[2])
new_val_shape=(int(encoded_X_val.shape[0]/20), 20, encoded_X_val.shape[1], encoded_X_val.shape[2])
new_test_shape=(int(encoded_X_test.shape[0]/20), 20, encoded_X_test.shape[1], encoded_X_test.shape[2])

# Initialize the new array with the target shape
reshaped_train = np.zeros(new_train_shape)
# Reshape using for loops
for i in range(new_train_shape[0]):  # Loop over the first dimension ()
    for j in range(new_train_shape[1]):  # Loop over the second dimension (20)
        # Calculate the corresponding index in the original array
        index = i * new_train_shape[1] + j
        reshaped_train[i, j] = encoded_X_train[index]

# Verify the shape
print(reshaped_train.shape)


# Initialize the new array with the target shape
reshaped_val = np.zeros(new_val_shape)
# Reshape using for loops
for i in range(new_val_shape[0]):  # Loop over the first dimension ()
    for j in range(new_val_shape[1]):  # Loop over the second dimension (20)
        # Calculate the corresponding index in the original array
        index = i * new_val_shape[1] + j
        reshaped_val[i, j] = encoded_X_val[index]

# Verify the shape
print(reshaped_val.shape)


# Initialize the new array with the target shape
reshaped_test = np.zeros(new_test_shape)
# Reshape using for loops
for i in range(new_test_shape[0]):  # Loop over the first dimension ()
    for j in range(new_test_shape[1]):  # Loop over the second dimension (20)
        # Calculate the corresponding index in the original array
        index = i * new_test_shape[1] + j
        reshaped_test[i, j] = encoded_X_test[index]

# Verify the shape
print(reshaped_test.shape)

In [ ]:
#reshape the arrays
from tensorflow.keras.layers import Reshape
reshape_output1 = Reshape((20,100*128))(reshaped_train)
x_tr=np.array(reshape_output1)

reshape_output2 = Reshape((20,100*128))(reshaped_val)
x_ts=np.array(reshape_output2)

reshape_output3 = Reshape((20,100*128))(reshaped_test)
x_vl=np.array(reshape_output3)

In [ ]:
#transform labels to numpy array
y_train_np = np.array(y_train)
y_val_np = np.array(y_val)
y_test_np = np.array(y_test)

In [ ]:
# classification
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM, Dense,Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

data_folder = '/content/drive/My Drive/Audios/Train'
classes = sorted(os.listdir(data_folder))

model = Sequential()
model.add(Flatten())
# Fully connected layer with 256 units
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))

# Fully connected layer with 64 units
model.add(Dense(64, activation='relu'))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x_tr, y_train_np, epochs=50, batch_size=32, validation_data=(x_ts, y_val_np))

loss, accuracy = model.evaluate(x_vl, y_test_np)
# Predictions and confusion matrix
y_pred_prob = model.predict(x_vl)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
f1 = f1_score(y_test_np, y_pred)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test_np, y_pred)
print('Confusion Matrix')
print(conf_matrix)

print('Classification Report')
print(classification_report(y_test_np, y_pred, target_names=classes))

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss graph
# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#classification with 1 LSTM block
model = Sequential()
model.add(LSTM(128, input_shape=(x_tr.shape[1], x_tr.shape[2]), return_sequences=False))

# Fully connected layer with 256 units
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))

# Fully connected layer with 64 units
model.add(Dense(64, activation='relu'))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x_tr, y_train_np, epochs=50, batch_size=32, validation_data=(x_ts, y_val_np), callbacks=[EarlyStopping(monitor='val_loss', patience=7)])

loss, accuracy = model.evaluate(x_vl, y_test_np)
# Predictions and confusion matrix
y_pred_prob = model.predict(x_vl)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
f1 = f1_score(y_test_np, y_pred)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


# Plot confusion matrix
conf_matrix = confusion_matrix(y_test_np, y_pred)
print('Confusion Matrix')
print(conf_matrix)

print('Classification Report')
print(classification_report(y_test_np, y_pred, target_names=classes))

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss graph
# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Classification with 2 LSTM blocks
model = Sequential()
model.add(LSTM(128, input_shape=(x_tr.shape[1], x_tr.shape[2]), return_sequences=True))
model.add(LSTM(128,return_sequences=False))

# Fully connected layer with 256 units
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))

# Fully connected layer with 64 units
model.add(Dense(64, activation='relu'))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x_tr, y_train_np, epochs=50, batch_size=32, validation_data=(x_ts, y_val_np), callbacks=[EarlyStopping(monitor='val_loss', patience=7)])

loss, accuracy = model.evaluate(x_vl, y_test_np)
# Predictions and confusion matrix
y_pred_prob = model.predict(x_vl)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
f1 = f1_score(y_test_np, y_pred)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test_np, y_pred)
print('Confusion Matrix')
print(conf_matrix)

print('Classification Report')
print(classification_report(y_test_np, y_pred, target_names=classes))

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Lass graph
# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Classification with 3 LSTM blocks
model = Sequential()
model.add(LSTM(128, input_shape=(x_tr.shape[1], x_tr.shape[2]), return_sequences=True))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128,return_sequences=False))

# Fully connected layer with 256 units
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))

# Fully connected layer with 64 units
model.add(Dense(64, activation='relu'))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x_tr, y_train_np, epochs=50, batch_size=32, validation_data=(x_ts, y_val_np), callbacks=[EarlyStopping(monitor='val_loss', patience=7)])

loss, accuracy = model.evaluate(x_vl, y_test_np)
# Predictions and confusion matrix
y_pred_prob = model.predict(x_vl)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
f1 = f1_score(y_test_np, y_pred)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test_np, y_pred)
print('Confusion Matrix')
print(conf_matrix)

print('Classification Report')
print(classification_report(y_test_np, y_pred, target_names=classes))

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss graph
# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# **SAE**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM, Dense,Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
classes = sorted(os.listdir(train_folder))

# In order the AE to process the signals easily, frame the signals into 25 ms
frame_length = 400 #25 ms window size
train_frames = [librosa.util.frame(signal, frame_length=frame_length, hop_length=frame_length).T for signal in train_signals]
val_frames = [librosa.util.frame(signal, frame_length=frame_length, hop_length=frame_length).T for signal in validation_signals]
test_frames = [librosa.util.frame(signal, frame_length=frame_length, hop_length=frame_length).T for signal in test_signals]

# Flatten frames for autoencoder
flat_X_train = np.concatenate(train_frames, axis=0)
flat_X_val = np.concatenate(val_frames, axis=0)
flat_X_test = np.concatenate(test_frames, axis=0)

y_train=train_labels
y_val=validation_labels
y_test=test_labels

# Define the autoencoder
input_dim = flat_X_train.shape[1]
latent_dim = 64

inputs = Input(shape=(input_dim,))
encoded = Dense(256, activation='relu')(inputs)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(latent_dim, activation='relu')(encoded)

decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)

autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(flat_X_train, flat_X_train, epochs=100, batch_size=32, shuffle=True, validation_data=(flat_X_val, flat_X_val))

encoder = Model(inputs, encoded)
# Save the model
autoencoder.save('/content/drive/My Drive/Results/Models/Sae.h5')
encoder.save('/content/drive/My Drive/Results/Models/SaeEncoder.h5')

In [ ]:
encoded_X_train=encoder.predict(flat_X_train)
encoded_X_val=encoder.predict(flat_X_val)
encoded_X_test=encoder.predict(flat_X_test)

import pickle

with open('/content/drive/My Drive/Results/Models/SaeTrainEncoded.pkl', 'wb') as f:
    pickle.dump((encoded_X_train, y_train), f)
with open('/content/drive/My Drive/Results/Models/SaeValEncoded.pkl', 'wb') as f:
    pickle.dump((encoded_X_val, y_val), f)
with open('/content/drive/My Drive/Results/Models/SaeTestEncoded.pkl', 'wb') as f:
    pickle.dump((encoded_X_test, y_test), f)


In [ ]:
encoded_X_train.shape
encoded_X_val.shape
encoded_X_test.shape

In [ ]:
new_train_shape=(int(encoded_X_train.shape[0]/20), 20, encoded_X_train.shape[1])
new_val_shape=(int(encoded_X_val.shape[0]/20), 20, encoded_X_val.shape[1])
new_test_shape=(int(encoded_X_test.shape[0]/20), 20, encoded_X_test.shape[1])

# Initialize the new array with the target shape
reshaped_train = np.zeros(new_train_shape)

# Reshape using for loops
for i in range(new_train_shape[0]):  # Loop over the first dimension ()
    for j in range(new_train_shape[1]):  # Loop over the second dimension (20)
        # Calculate the corresponding index in the original array
        index = i * new_train_shape[1] + j
        reshaped_train[i, j] = encoded_X_train[index]

# Verify the shape
print(reshaped_train.shape)

# Initialize the new array with the target shape
reshaped_val = np.zeros(new_val_shape)

# Reshape using for loops
for i in range(new_val_shape[0]):  # Loop over the first dimension ()
    for j in range(new_val_shape[1]):  # Loop over the second dimension (20)
        # Calculate the corresponding index in the original array
        index = i * new_val_shape[1] + j
        reshaped_val[i, j] = encoded_X_val[index]

# Verify the shape
print(reshaped_val.shape)

# Initialize the new array with the target shape
reshaped_test = np.zeros(new_test_shape)

# Reshape using for loops
for i in range(new_test_shape[0]):  # Loop over the first dimension ()
    for j in range(new_test_shape[1]):  # Loop over the second dimension (20)
        # Calculate the corresponding index in the original array
        index = i * new_test_shape[1] + j
        reshaped_test[i, j] = encoded_X_test[index]

# Verify the shape
print(reshaped_test.shape)

In [ ]:
# Calssification with 0 LSTM
model = Sequential()
model.add(Flatten())
# Fully connected layer with 256 units
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))

# Fully connected layer with 64 units
model.add(Dense(64, activation='relu'))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(reshaped_train, y_train_np, epochs=50, batch_size=32, validation_data=(reshaped_val, y_val_np), callbacks=[EarlyStopping(monitor='val_loss', patience=7)])

loss, accuracy = model.evaluate(reshaped_test, y_test_np)
# Predictions and confusion matrix
y_pred_prob = model.predict(reshaped_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
f1 = f1_score(y_test_np, y_pred)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test_np, y_pred)
print('Confusion Matrix')
print(conf_matrix)

print('Classification Report')
print(classification_report(y_test_np, y_pred, target_names=classes))

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss graph
# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Calssification with 1 LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(reshaped_val.shape[1], reshaped_val.shape[2]), return_sequences=False))

# Fully connected layer with 256 units
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))

# Fully connected layer with 64 units
model.add(Dense(64, activation='relu'))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(reshaped_train, y_train_np, epochs=50, batch_size=32, validation_data=(reshaped_val, y_val_np), callbacks=[EarlyStopping(monitor='val_loss', patience=7)])

loss, accuracy = model.evaluate(reshaped_test, y_test_np)
# Predictions and confusion matrix
y_pred_prob = model.predict(reshaped_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
f1 = f1_score(y_test_np, y_pred)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test_np, y_pred)
print('Confusion Matrix')
print(conf_matrix)

print('Classification Report')
print(classification_report(y_test_np, y_pred, target_names=classes))

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss graph
# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Calssification with 2 LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(reshaped_val.shape[1], reshaped_val.shape[2]), return_sequences=True))
model.add(LSTM(128,return_sequences=False))

# Fully connected layer with 256 units
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))

# Fully connected layer with 64 units
model.add(Dense(64, activation='relu'))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(reshaped_train, y_train_np, epochs=50, batch_size=32, validation_data=(reshaped_val, y_val_np), callbacks=[EarlyStopping(monitor='val_loss', patience=7)])

loss, accuracy = model.evaluate(reshaped_test, y_test_np)
# Predictions and confusion matrix
y_pred_prob = model.predict(reshaped_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
f1 = f1_score(y_test_np, y_pred)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test_np, y_pred)
print('Confusion Matrix')
print(conf_matrix)

print('Classification Report')
print(classification_report(y_test_np, y_pred, target_names=classes))

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss graph
# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Calssification with 3 LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(reshaped_val.shape[1], reshaped_val.shape[2]), return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128,return_sequences=False))

# Fully connected layer with 256 units
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))

# Fully connected layer with 64 units
model.add(Dense(64, activation='relu'))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(reshaped_train, y_train_np, epochs=50, batch_size=32, validation_data=(reshaped_val, y_val_np), callbacks=[EarlyStopping(monitor='val_loss', patience=7)])

loss, accuracy = model.evaluate(reshaped_test, y_test_np)
# Predictions and confusion matrix
y_pred_prob = model.predict(reshaped_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
f1 = f1_score(y_test_np, y_pred)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test_np, y_pred)
print('Confusion Matrix')
print(conf_matrix)

print('Classification Report')
print(classification_report(y_test_np, y_pred, target_names=classes))

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss graph
# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# **RAE**

In [ ]:
classes = sorted(os.listdir(train_folder))

In [ ]:
classes = sorted(os.listdir(train_folder))

#in order the AE to process the signals easily, frame the signals into 25 ms
frame_length = 400 #25 ms window size
train_frames = [librosa.util.frame(signal, frame_length=frame_length, hop_length=frame_length).T for signal in train_signals]
val_frames = [librosa.util.frame(signal, frame_length=frame_length, hop_length=frame_length).T for signal in validation_signals]
test_frames = [librosa.util.frame(signal, frame_length=frame_length, hop_length=frame_length).T for signal in test_signals]

#Flatten frames for autoencoder
flat_X_train = np.concatenate(train_frames, axis=0)
flat_X_val = np.concatenate(val_frames, axis=0)
flat_X_test = np.concatenate(test_frames, axis=0)

y_train=train_labels
y_val=validation_labels
y_test=test_labels

In [ ]:
# Reshape for Conv1D input (add channel dimension)
from tensorflow.keras.layers import UpSampling1D
flat_X_train_reshaped = flat_X_train.reshape((flat_X_train.shape[0], flat_X_train.shape[1], 1))
flat_X_val_reshaped = flat_X_val.reshape((flat_X_val.shape[0], flat_X_val.shape[1], 1))
flat_X_test_reshaped = flat_X_test.reshape((flat_X_test.shape[0], flat_X_test.shape[1], 1))

# Define input shape based on reshaped data
input_shape = (flat_X_train_reshaped.shape[1], flat_X_train_reshaped.shape[2]) # (sequence_length, num_features)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Dropout, Input, RepeatVector
from tensorflow.keras.optimizers import Adam

latent_dim = 32

# Encoder
inputs = Input(shape=input_shape)
encoded = LSTM(128, return_sequences=True, implementation=2)(inputs)
encoded = Dropout(0.2)(encoded)
encoded = LSTM(64, return_sequences=False, implementation=2)(encoded)  # Return sequences=False for Dense layer
encoded = Dropout(0.2)(encoded)
encoded = Dense(latent_dim, activation='relu')(encoded)

# Decoder
decoded = RepeatVector(input_shape[0])(encoded)  # Repeat to match the sequence length
decoded = LSTM(64, return_sequences=True, implementation=2)(decoded)
decoded = Dropout(0.2)(decoded)
decoded = LSTM(128, return_sequences=True, implementation=2)(decoded)
decoded = Dropout(0.2)(decoded)
decoded = TimeDistributed(Dense(input_shape[1]))(decoded)  # Adjusted to match the number of features

# Models
autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)

# Compile the model
autoencoder.compile(optimizer=Adam(), loss='mse')

# Ensure you are using a GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Train the model
batch_size = 32
epochs = 100

autoencoder.fit(flat_X_train_reshaped, flat_X_train_reshaped, batch_size=batch_size, epochs=epochs, validation_data=(flat_X_val_reshaped,flat_X_val_reshaped))

In [ ]:
autoencoder.save('/content/drive/My Drive/Results/Models/Rae.h5')


encoded_X_train = [encoder.predict(flat_X_train_reshaped)]
encoded_X_val = [encoder.predict(flat_X_val_reshaped)]
encoded_X_test = [encoder.predict(flat_X_test_reshaped)]

import pickle

with open('/content/drive/My Drive/tezSonrasi/RAEtrain23082024.pkl', 'wb') as f:
    pickle.dump((encoded_X_train, y_train), f)
with open('/content/drive/My Drive/tezSonrasi/RAEval23082024.pkl', 'wb') as f:
    pickle.dump((encoded_X_val, y_val), f)
with open('/content/drive/My Drive/tezSonrasi/RAEtest23082024.pkl', 'wb') as f:
    pickle.dump((encoded_X_test, y_test), f)

In [ ]:
encoder =Model(inputs, encoded)

encoder.save('/content/drive/My Drive/Results/Models/RaeEncoder.h5')

In [ ]:
#encoded_X_train/val/test has 1 item in the list, so take the first and only element's shape
encoded_X_train[0].shape

In [ ]:
encoded_X_val[0].shape

In [ ]:

encoded_X_test[0].shape

In [ ]:
encoded_X_train_np=np.array(encoded_X_train[0])
encoded_X_val_np=np.array(encoded_X_val[0])
encoded_X_test_np=np.array(encoded_X_test[0])

In [ ]:
train_new_shape = (int((encoded_X_train[0].shape[0])/20),20,encoded_X_train[0].shape[1])

# Initialize the new array with the target shape
reshaped_train = np.zeros(train_new_shape)

# Reshape using for loops
for i in range(train_new_shape[0]):  # Loop over the first dimension ()
    for j in range(train_new_shape[1]):  # Loop over the second dimension (20)
        # Calculate the corresponding index in the original array
        index = i * train_new_shape[1] + j
        reshaped_train[i, j] = encoded_X_train_np[index]

# Verify the shape
print(reshaped_train.shape)

val_new_shape = (int((encoded_X_val[0].shape[0])/20),20,encoded_X_val[0].shape[1])

# Initialize the new array with the target shape
reshaped_val = np.zeros(val_new_shape)

# Reshape using for loops
for i in range(val_new_shape[0]):  # Loop over the first dimension ()
    for j in range(val_new_shape[1]):  # Loop over the second dimension (20)
        # Calculate the corresponding index in the original array
        index = i * val_new_shape[1] + j
        reshaped_val[i, j] = encoded_X_val_np[index]

# Verify the shape
print(reshaped_val.shape)

test_new_shape = (int((encoded_X_test[0].shape[0])/20),20,encoded_X_test[0].shape[1])

# Initialize the new array with the target shape
reshaped_test = np.zeros(test_new_shape)

# Reshape using for loops
for i in range(test_new_shape[0]):  # Loop over the first dimension ()
    for j in range(test_new_shape[1]):  # Loop over the second dimension (20)
        # Calculate the corresponding index in the original array
        index = i * test_new_shape[1] + j
        reshaped_test[i, j] = encoded_X_test_np[index]

# Verify the shape
print(reshaped_test.shape)

In [ ]:
y_train_np=np.array(y_train)
y_val_np=np.array(y_val)
y_test_np=np.array(y_test)

In [ ]:
# Classification with 0 LSTM
model = Sequential()
model.add(Flatten())
# Fully connected layer with 256 units
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))

# Fully connected layer with 64 units
model.add(Dense(64, activation='relu'))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(reshaped_train, y_train_np, epochs=50, batch_size=32, validation_data=(reshaped_val, y_val_np), callbacks=[EarlyStopping(monitor='val_loss', patience=7)])

loss, accuracy = model.evaluate(reshaped_test, y_test_np)
# Predictions and confusion matrix
y_pred_prob = model.predict(reshaped_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
f1 = f1_score(y_test_np, y_pred)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test_np, y_pred)
print('Confusion Matrix')
print(conf_matrix)

print('Classification Report')
print(classification_report(y_test_np, y_pred, target_names=classes))

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss graph
# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Classification with 1 LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(reshaped_val.shape[1], reshaped_val.shape[2]), return_sequences=False))

# Fully connected layer with 256 units
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))

# Fully connected layer with 64 units
model.add(Dense(64, activation='relu'))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(reshaped_train, y_train_np, epochs=50, batch_size=32, validation_data=(reshaped_val, y_val_np), callbacks=[EarlyStopping(monitor='val_loss', patience=7)])

loss, accuracy = model.evaluate(reshaped_test, y_test_np)
# Predictions and confusion matrix
y_pred_prob = model.predict(reshaped_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
f1 = f1_score(y_test_np, y_pred)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test_np, y_pred)
print('Confusion Matrix')
print(conf_matrix)

print('Classification Report')
print(classification_report(y_test_np, y_pred, target_names=classes))

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss graph
# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Classification with 2 LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM, Dense,Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

model = Sequential()
model.add(LSTM(128, input_shape=(reshaped_val.shape[1], reshaped_val.shape[2]), return_sequences=True))
model.add(LSTM(128,return_sequences=False))

# Fully connected layer with 256 units
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))

# Fully connected layer with 64 units
model.add(Dense(64, activation='relu'))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(reshaped_train, y_train_np, epochs=50, batch_size=32, validation_data=(reshaped_val, y_val_np), callbacks=[EarlyStopping(monitor='val_loss', patience=7)])

loss, accuracy = model.evaluate(reshaped_test, y_test_np)
# Predictions and confusion matrix
y_pred_prob = model.predict(reshaped_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
f1 = f1_score(y_test_np, y_pred)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test_np, y_pred)
print('Confusion Matrix')
print(conf_matrix)

print('Classification Report')
print(classification_report(y_test_np, y_pred, target_names=classes))

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss graph
# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Classification with 3 LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(reshaped_val.shape[1], reshaped_val.shape[2]), return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128,return_sequences=False))

# Fully connected layer with 256 units
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))

# Fully connected layer with 64 units
model.add(Dense(64, activation='relu'))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(reshaped_train, y_train_np, epochs=50, batch_size=32, validation_data=(reshaped_val, y_val_np), callbacks=[EarlyStopping(monitor='val_loss', patience=7)])

loss, accuracy = model.evaluate(reshaped_test, y_test_np)
# Predictions and confusion matrix
y_pred_prob = model.predict(reshaped_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
f1 = f1_score(y_test_np, y_pred)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test_np, y_pred)
print('Confusion Matrix')
print(conf_matrix)

print('Classification Report')
print(classification_report(y_test_np, y_pred, target_names=classes))

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#loss grafiği
# Plot training and validation accuracy curves
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()